For regularization refer to this
https://github.com/vvshankar78/DeepLearning/blob/master/Extensive%20VisionAI%20(EVA5)/06_Regularization%20%26%20Modular%20Coding/04_Modular%2Bregularization_FINAL.ipynb


for batch norm modular coding refer to this
https://github.com/vvshankar78/DeepLearning/blob/master/Extensive%20VisionAI%20(EVA5)/06_Regularization%20%26%20Modular%20Coding/02_visualize_misclassified.ipynb



In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [ ]:
# Train Phase transformations
train_transforms = transforms.Compose([
                                      #transforms.ToPILImage(),
                                      #transforms.Resize((28, 28)),
                                      #transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       #transforms.RandomRotation((-7.0, 7.0), fill=(0.13,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)), # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values. 
                                       # Note the difference between (0.1307) and (0.1307,)
                                       #transforms.RandomRotation((-5.0, 5.0), fill=(-0.42,)),
                                       transforms.RandomAffine((-5,5), translate=None, scale=None, shear=5, resample=False, fillcolor=(-0.42,))
                                       ])

# Test Phase transformations
test_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)

SEED = 1

# CUDA?
cuda = torch.cuda.is_available()
print("CUDA Available?", cuda)

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

# dataloader arguments - something you'll fetch these from cmdprmt
dataloader_args = dict(shuffle=True, batch_size=64, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1315: UserWarning: Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead
  "Argument resample is deprecated and will be removed since v0.10.0. Please, use interpolation instead"
/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:1329: UserWarning: Argument fillcolor is deprecated and will be removed since v0.10.0. Please, use fill instead
  "Argument fillcolor is deprecated and will be removed since v0.10.0. Please, use fill instead"



Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...
Done!
CUDA Available? True


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid poten

In [ ]:
# from tqdm.notebook import tqdm

#Use scheduler
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    # pbar = tqdm(train_loader)
    train_loss = 0
    correct = 0
    num_loops = 0
    criterion = nn.CrossEntropyLoss()
    # for batch_idx, (data, target) in enumerate(pbar):
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        # loss = F.nll_loss(output, target)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        # pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        train_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        num_loops +=1
 
    train_loss /= num_loops
    # scheduler.step()
    # scheduler.step(train_loss)
    hist_train_loss.append(train_loss)
    hist_train_acc.append(100. * correct / len(train_loader.dataset))
    print("Training Average loss: {:.6f}, Accuracy = ({:.6f}%)".format(train_loss, 100. * correct / len(train_loader.dataset)))

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    hist_test_loss.append(test_loss)
    hist_test_acc.append(100. * correct / len(test_loader.dataset))

    print('Test set: Average loss: {:.6f}, Accuracy: {}/{} ({:.6f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = [15, 6]

def training_curves():
  plt.subplot(1,2, 1)

  plt.plot(np.array(hist_test_acc))
  plt.plot(np.array(hist_train_acc), 'r')
  plt.legend(["test_acc", "train_acc"])
  plt.title("Accuracy per epoch")

  plt.subplot(1,2,2)
  plt.plot(hist_test_loss)
  plt.plot(hist_train_loss, 'r')
  plt.legend(["test_loss", "train_loss"])
  plt.title("Loss per epoch")

### These functions will go to Utils

In [ ]:
def Gen_BN(out_features, channel_size, norm_type):
  if norm_type=="BN":
    m = nn.BatchNorm2d(out_features)
  elif norm_type == "LN":
    # x = torch.rand(2,out_features,channel_size,channel_size)
    # m = nn.LayerNorm(x.size()[1:])
    m = nn.LayerNorm((out_features, channel_size, channel_size))
  return m

def conv_output_shape(h_w, kernel_size=1, stride=1, pad=0, dilation=1):
    """
    Utility function for computing output of convolutions
    takes a tuple of (h,w) and returns a tuple of (h,w)
    """
    
    if type(h_w) is not tuple:
        h_w = (h_w, h_w)
    
    if type(kernel_size) is not tuple:
        kernel_size = (kernel_size, kernel_size)
    
    if type(stride) is not tuple:
        stride = (stride, stride)
    
    if type(pad) is not tuple:
        pad = (pad, pad)
    
    h = (h_w[0] + (2 * pad[0]) - (dilation * (kernel_size[0] - 1)) - 1)// stride[0] + 1
    w = (h_w[1] + (2 * pad[1]) - (dilation * (kernel_size[1] - 1)) - 1)// stride[1] + 1
    
    return h, w

x, y = conv_output_shape(28, 3, 1,0,1)
x, y
temp = Gen_BN(12, 28, "LN")
temp

LayerNorm((12, 28, 28), eps=1e-05, elementwise_affine=True)

This class will take in the optin of which normalization we would want to use.. 

Group normalization is pending


Input to the architecture is the nrmalization type and input image size

In [ ]:
dropout_value = 0.05


class Net2(nn.Module):
    def get_bn(self, out_features, channel_size, norm_type):
        if norm_type=="BN":
          m = nn.BatchNorm2d(out_features)
        elif norm_type == "LN":
          m = nn.LayerNorm((out_features, channel_size, channel_size))
        return m

    def conv_block(self, in_features, out_features, kernel_size, input_channel_size, Norm_type,pading=0, last_block=True):
      # convolution
      layers = []
      layers = [nn.Conv2d(in_features, out_features, kernel_size, padding=pading, bias=False), nn.ReLU()]
      out_img_size, _ = conv_output_shape(input_channel_size, kernel_size, stride=1,pad=pading,dilation=1) # get the image size after convolution

      #Generate Normalization
      norm_list = ['BN', 'LN'] # this is the list of normalization that is allowed. 
      if (Norm_type in norm_list) and (kernel_size != 1) and (last_block):   # Check if normalization is in the list, or if its not 1x1 or if its not last block. 
        BN = self.get_bn(out_features, out_img_size, Norm_type)
        layers.append(BN)

      # Add Dropout - Check if its not last block or 1x1 convolution.   
      if kernel_size !=1 and last_block :
        layers.append(nn.Dropout(dropout_value))
      block = nn.Sequential(*layers)
      return block, out_img_size

    def max_pool_block(self, kernal_size, stride, img_size):
        pool = nn.MaxPool2d(kernal_size, stride) # output_size = 13
        out_img_size,_ = conv_output_shape(img_size,kernal_size, stride)
        return pool, out_img_size

    def __init__(self, Norm_type, input_img_size=(1,28,28)):
        super(Net2, self).__init__()
        self.convblock1, img_size = self.conv_block(1,8,3,input_img_size[1],Norm_type) # input kernal, output_kernals, convolution, input image size, padding, normalization type #26
        self.convblock2, img_size = self.conv_block(8,16,3,img_size,Norm_type) #24
        self.pool1, img_size = self.max_pool_block(2,2, img_size)  #12
        self.convblock4, img_size = self.conv_block(16,8,1,img_size,Norm_type) #12
        self.convblock5, img_size = self.conv_block(8,16,3,img_size,Norm_type) #10
        self.convblock6, img_size = self.conv_block(16,20,3,img_size,Norm_type) #8
        self.convblock7, img_size = self.conv_block(20,8,1,img_size,Norm_type) #8
        self.convblock8, img_size = self.conv_block(8,16,3,img_size,Norm_type, last_block=False) #6

        self.gap = nn.Sequential(nn.AvgPool2d(kernel_size=6)) # output_size = 1
        self.fc1 = nn.Linear(16, 10)

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        #x = self.convblock3(x)
        x = self.pool1(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = self.convblock6(x)
        x = self.convblock7(x)
        x = self.convblock8(x)
        x = self.gap(x)
        x = x.view(-1, x.shape[1]*x.shape[2]*x.shape[3])
        x = self.fc1(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

model = Net2("BN", input_img_size=(1, 28, 28)).to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 26, 26]              72
              ReLU-2            [-1, 8, 26, 26]               0
       BatchNorm2d-3            [-1, 8, 26, 26]              16
           Dropout-4            [-1, 8, 26, 26]               0
            Conv2d-5           [-1, 16, 24, 24]           1,152
              ReLU-6           [-1, 16, 24, 24]               0
       BatchNorm2d-7           [-1, 16, 24, 24]              32
           Dropout-8           [-1, 16, 24, 24]               0
         MaxPool2d-9           [-1, 16, 12, 12]               0
           Conv2d-10            [-1, 8, 12, 12]             128
             ReLU-11            [-1, 8, 12, 12]               0
           Conv2d-12           [-1, 16, 10, 10]           1,152
             ReLU-13           [-1, 16, 10, 10]               0
      BatchNorm2d-14           [-1, 16,

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import ExponentialLR

hist_test_loss = []
hist_test_acc = []
hist_train_loss = []
hist_train_acc = []

optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9,)

scheduler =ReduceLROnPlateau(optimizer=optimizer, patience=2, verbose=True)

for epoch in range(1, 4):
    print("Epoch: ", epoch)
    train(model, device, train_loader, optimizer, epoch)
    print("learning rate", optimizer.param_groups[0]['lr'])
    test(model, device, test_loader)


Epoch:  1


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Training Average loss: 0.061735, Accuracy = (98.105000%)
learning rate 0.1
Test set: Average loss: 0.059488, Accuracy: 9816/10000 (98.160000%)

Epoch:  2
Training Average loss: 0.056768, Accuracy = (98.268333%)
learning rate 0.1
Test set: Average loss: 0.031461, Accuracy: 9900/10000 (99.000000%)

Epoch:  3
Training Average loss: 0.049957, Accuracy = (98.491667%)
learning rate 0.1
Test set: Average loss: 0.044592, Accuracy: 9852/10000 (98.520000%)



In [ ]:
input_size=(1, 28, 28)